In [86]:
# load the model encoder data
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions as dist
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torchtext.data.utils import get_tokenizer
import torch
import torch.nn as nn
import numpy as np
import embedding
from embedding import d_model

num_heads = 8   # number of attention heads
d_ff = 128      # dimension of feedforward network
num_layers = 6  # ff network layers

with open("chapter1.txt", "r", encoding="utf-8") as f:
    text = f.read()
vectors, vocabulary = embedding.get_vectors(text)

encoder_output = torch.load('encoded_vector.pth', weights_only=False)
print(encoder_output.shape)



['<unk>', '<pad>', 'the', ',', 'and', '.', 'a', 'was', 'she', 'her', 'in', 'of', 'to', 'dorothy', 'it', 'at', 'had', 'house', 'that', 'as', 'from', 'toto', 'gray', 'aunt', 'but', 'em', 'he', 'they', 'could', 'not', 'there', 'were', 'wind', 'cyclone', 'door', 'floor', 'henry', 'him', 'one', 'uncle', 'when', 'with', 'came', 'down', 'for', 'great', 'his', 'hole', 'little', 'long', 'looked', 'now', 'room', 'small', 'so', 'up', 'air', 'all', 'around', 'bed', 'by', 'eyes', 'first', 'into', 'on', 'see', 'stood', 'sun', 'trap', 'upon', 'where', 'would', '”', 'after', 'also', 'away', 'be', 'coming', 'dark', 'fell', 'felt', 'girl', 'grass', 'if', 'made', 'miles', 'no', 'quite', 'reached', 'sat', 'side', 'sky', 'still', 'through', 'too', 'what', 'which', 'who', 'wife', '!', 'about', 'again', 'arms', 'badly', 'been', 'black', 'called', 'carried', 'caught', 'cellar', 'corner', 'did', 'dishes', 'dorothy’s', 'ears', 'easily', 'even', 'every', 'four', 'go', 'got', 'hand', 'happen', 'happened', 'hard',

In [87]:
decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=num_heads, batch_first=True)
decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)

def greedy_decode(encoder_output, tgt_embeddings, decoder, start_token_id, d_model, max_length, vocab_size):
    final_linear = nn.Linear(d_model, vocab_size)
    tgt_tokens = torch.full((1, 1), start_token_id)  # initialize with <SOS>
    generated = [tgt_tokens]   # store token indices step-by-step
    
    for step in range(max_length):
        tgt_embedded = tgt_embeddings(tgt_tokens)  # [batch_size, cur_length, d_model]
        tgt_mask = torch.nn.Transformer.generate_square_subsequent_mask(tgt_embedded.shape[1])
        out = decoder(tgt=tgt_embedded, memory=encoder_output, tgt_mask=tgt_mask)
        
        # Project to vocab and get next token
        logits = final_linear(out[:, -1, :])   # [batch_size, vocab_size]; use last token's output
        next_token = torch.argmax(logits, dim=-1, keepdim=True)  # [batch_size, 1]
        
        print(3,next_token)
        
        tgt_tokens = torch.cat([tgt_tokens, next_token], dim=1)  # append new token to the input
        generated.append(next_token)
        
        # You may want to stop if '<EOS>' is generated for all batch samples
        if next_token.item() == vocabulary['<EOS>']:
            break

    return tgt_tokens




In [88]:
vocab_size = vocabulary.__len__()
tgt_embeddings = nn.Embedding(vocab_size, d_model)


target_string = "<SOS> aunt em was <EOS>"
target_tokens = embedding.tokenize(target_string)
token_ids = [vocabulary[token] for token in target_tokens]
t = torch.tensor(token_ids).unsqueeze(0)

emb = nn.Embedding(vocab_size, embedding_dim=d_model)
tgt_embedding = emb(t)
print('z',tgt_embedding.shape)


start_token_id = vocabulary['<SOS>']
print('a',start_token_id)
print('b',encoder_output.shape)
result = greedy_decode(encoder_output, tgt_embeddings, decoder, start_token_id,d_model, max_length=100, vocab_size=vocab_size)
print(result)


print(result.shape)
for tokens in result[0:]:
    for token in tokens:
        print(token)

# Suppose your decoder result is a tensor of token indices
r = result.squeeze().tolist()
print(r)

print(vocabulary.itos)



z torch.Size([1, 5, 16])
a 0
b torch.Size([1, 1279, 16])
3 tensor([[145]])
3 tensor([[139]])
3 tensor([[176]])
3 tensor([[51]])
3 tensor([[381]])
3 tensor([[371]])
3 tensor([[50]])
3 tensor([[283]])
3 tensor([[194]])
3 tensor([[277]])
3 tensor([[213]])
3 tensor([[50]])
3 tensor([[283]])
3 tensor([[196]])
3 tensor([[0]])
tensor([[  0, 145, 139, 176,  51, 381, 371,  50, 283, 194, 277, 213,  50, 283,
         196,   0]])
torch.Size([1, 16])
tensor(0)
tensor(145)
tensor(139)
tensor(176)
tensor(51)
tensor(381)
tensor(371)
tensor(50)
tensor(283)
tensor(194)
tensor(277)
tensor(213)
tensor(50)
tensor(283)
tensor(196)
tensor(0)
[0, 145, 139, 176, 51, 381, 371, 50, 283, 194, 277, 213, 50, 283, 196, 0]
['<unk>', '<pad>', 'the', ',', 'and', '.', 'a', 'was', 'she', 'her', 'in', 'of', 'to', 'dorothy', 'it', 'at', 'had', 'house', 'that', 'as', 'from', 'toto', 'gray', 'aunt', 'but', 'em', 'he', 'they', 'could', 'not', 'there', 'were', 'wind', 'cyclone', 'door', 'floor', 'henry', 'him', 'one', 'uncle',